In [1]:
%gui asyncio

In [2]:
import asyncio
import pandas as pd
import numpy as np
import html

In [3]:
from ipywidgets import widgets as w
from IPython.display import clear_output
import time 

def gen_table(numRows):
    rows = []
    reset_opt = ''
    
    done = w.Button(
        description='Done',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click if done with annotation',
        icon='check' # (FontAwesome names without the `fa-` prefix)
    )
    
    gs = w.GridspecLayout(numRows+1, 1)
    gs[numRows, 0] = done
    

    for i in range(numRows):
        argumentative = w.ToggleButtons(
            options=[0, 1, 'lacks context'],
            description='Argumentative:',
            disabled=False,
            button_style='',
        )

        evidence = w.ToggleButtons(
            options=[0, 1],
            description='Evidence:',
            disabled=False,
            button_style='',
        )

        claim = w.ToggleButtons(
            options=[0, 1],
            description='Claim:',
            disabled=False,
            button_style='',
        )

        procon = w.ToggleButtons(
            options=[-1, 0, 1],
            description='Pro/Con:',
            disabled=False,
            button_style='',
        )

        arg_type = w.ToggleButtons(
            options=[reset_opt, 'implicit', 'explicit', 'unrelated'],
            description='Argument type:',
            disabled=False,
            button_style='',
        )

        evidence_type = w.ToggleButtons(
            options=[reset_opt, 'expert', 'anecdotal', 'study'],
            description='Evidence type:',
            disabled=False,
            button_style='',
        )
        
        gsl = w.GridspecLayout(4, 2)
        
        topic = w.HTML()
        tweet = w.HTML()
        vector = {'value': ''} # This happens to save the tweet vector
        rows.append((topic, tweet, argumentative, claim, evidence, procon, arg_type, evidence_type, {'row_index': 0}))
        
        gs[i, 0] = gsl
        
        gsl[0, 0] = topic
        gsl[0, 1] = tweet
        gsl[1, 0] = argumentative
        gsl[1, 1] = claim
        gsl[2, 0] = evidence
        gsl[2, 1] = procon
        gsl[3, 0] = arg_type
        gsl[3, 1] = evidence_type
    
    return gs, rows, done


def annotate(df, num_rows = 10, annotation_file = 'sample_annotated.csv'):
    gs, rows, done = gen_table(num_rows)
    reset_opt = ''
    counter = 0
    
    out = w.HTML()

            
    def combined(_):
        save()
        iterate()

    def save():
        nonlocal rows
        nonlocal df
        nonlocal annotation_file
        
        for (topic, tweet, argumentative, claim, evidence, procon, arg_type, evidence_type, data) in rows:
            i = data['row_index']
            df.at[i, 'evidence'] = evidence.value
            df.at[i, 'claim'] = claim.value
            df.at[i, 'procon'] = procon.value
            df.at[i, 'arg_type'] = arg_type.value
            df.at[i, 'argumentative'] = argumentative.value
            df.at[i, 'evidence_type'] = evidence_type.value

        df.to_csv(annotation_file, index=False)
        
    def iterate():
        nonlocal counter
        nonlocal rows
        nonlocal df
        nonlocal reset_opt
        
        for i, ((topic, tweet, argumentative, claim, evidence, procon, arg_type, evidence_type, data), row) in enumerate(zip(rows, df.iloc[counter:counter+len(rows)].iloc)):
            topic.value =  f'Tweet number: {i + counter + 1} <br/> Topic: {html.escape(row.topic)}' # Sets the tweet text
            tweet.value = html.escape(row.tweet)
            data['row_index'] = i + counter
            
            argumentative.value = 0
            evidence.value = 0
            claim.value = 0
            procon.value = 0
            arg_type.value = reset_opt
            evidence_type.value = reset_opt


        counter = counter + len(rows)

    done.on_click(combined)
    iterate() # We iterate once to pull in the first set of tweets
    return gs

In [4]:
df = pd.read_csv('sample.csv')
#test = pd.read_csv('test.csv')
#dev = pd.read_csv('dev.csv')
#df = df.loc[~df.tweet.isin(test['tweet'].values)]
#df = df.loc[~df.tweet.isin(dev['tweet'].values)]#.sample(100)

In [5]:
len(df)

75

In [6]:
sLength = len(df)
df = df.assign(argumentative=pd.Series(['']*sLength).values)
df = df.assign(evidence=pd.Series(np.zeros(sLength)).values)
df = df.assign(claim=pd.Series(np.zeros(sLength)).values)
df = df.assign(procon=pd.Series(np.zeros(sLength)).values)
df = df.assign(arg_type=pd.Series(['']*sLength).values)
df = df.assign(evidence_type=pd.Series(['']*sLength).values)
df = df.reset_index()

In [7]:
annotate(df, 5)

GridspecLayout(children=(Button(description='Done', icon='check', layout=Layout(grid_area='widget001'), style=…

In [8]:
list(df.iloc)

[index                                                            0
 tweet            When u rent shit out, u still need permission ...
 id                                                     1.51104e+18
 lang                                                            en
 topic            We should exempt meat production from carbon t...
 score                                                            0
 argumentative                                                     
 evidence                                                         0
 claim                                                            0
 procon                                                           0
 arg_type                                                          
 evidence_type                                                     
 Name: 0, dtype: object,
 index                                                            1
 tweet            <MENTION> <MENTION> 2/2 the cattle after all a...
 id                    